# Trabajo de Curso de Visión por Computador
## Autores:
- Héctor Wood Santana
- Alejandro Viera Ruiz 

### Notas:
#### Mano Derecha:
    - Índice = Ratón (Hecho) 
    - Pulgar + Corazón = Abrir Teclado Virtual Macro 1 & 3
    - Pulgar + Anular = Macro 2 & 4
    - Pulgar + Meñique = Click Central??? ¿Flag para cambiar modos y asi tener mas opciones en la mano izquierda? (Hecho)
#### Mano Izquierda:
    - Pulgar + Índice =  Click Izquierdo (Hecho)
    - Pulgar + Corazón = Click Derecho (Hecho) 
    - Pulgar + Anular = Macro 5 & 7
    - Pulgar + Meñique = Macro 6 & 8


### Tareas:
    1º  Hacer Mano derecha (Héctor)
    2º  Pensar en Macros para las 8 opciones de la maño izquierda (Alejandro y ¿Héctor?)
    3º  Buscar como abrir aplicaciones (Alejandro)
    4º  Mirar calculo de la distancia para los clicks relativa al bounding box de la mano (Héctor)
    5º  Hacer Memoria
    6º  ¿Hacer juego?


#### Comentarios & Observaciones:

- He probado que no reconoce más de 2 manos. En cualquier combinación.
- 2 Manos derechas hacen saltar el raton entre las dos posiciones.
- Libreria para abrir apps "subprocess".

##### Referencias:
- https://docs.python.org/3.13/library/subprocess.html#module-subprocess
- https://ai.google.dev/edge/mediapipe/solutions/vision/gesture_recognizer?hl=es-419
- https://ai.google.dev/edge/mediapipe/solutions/vision/hand_landmarker?hl=es-419
- https://pyautogui.readthedocs.io/en/latest/index.html
- https://psutil.readthedocs.io/en/latest/
- Chat GPT
- https://www.geeksforgeeks.org/how-to-check-if-an-application-is-open-in-python/
- https://docs.python.org/es/3.8/library/threading.html
- https://github.com/learncodebygaming/pydirectinput (Libreria PyDirectinput)


#### Entrega

Si bien se fijará una fecha de entrega final de cara cada convocatoria, durante las sesiones del 28 y 29 de noviembre, en las sesiones prácticas se presentará la propuesta inicial del trabajo, de cara a recibir comentarios y consideraciones por parte del equipo docente.

Señalar que de cara a la evaluación del trabajo final se tendrán en cuenta los siguientes aspectos y sus correspondientes ponderaciones:

    alcance y dificultad de la propuesta (45%)
    calidad de la memoria (20%)
    exposición y defensa (20%)
    material audiovisual (15%)

Con posterioridad a la fecha límite de entrega, preferiblemente coincidente con la fecha del examen de convocatoria, se llevará a cabo una presentación del trabajo (10'). En coherencia con los aspectos valorados, de cara a la entrega final por medio de un enlace a un repositorio, el material entregado debe ser completo, permitiendo la reproducibilidad del desarrollo. Se sugiere que contenga al menos los siguientes elementos:

    Documento de memoria
        Motivación/argumentación del trabajo
        Objetivo de la propuesta
        Descripción técnica del trabajo realizado
        Fuentes y tecnologías utilizadas
        Conclusiones y propuestas de ampliación
        Indicación de herramientas/tecnologías con las que les hubiera gustado contar
        Diario de reuniones del grupo
        Créditos materiales no originales del grupo
    Enlace al código fuente
    Imágenes de entrenamiento (en su caso)
    Imagen "carátula" del trabajo
    Vídeo resumen de venta del trabajo
    Todo aquel material que consideren de interés


In [1]:
import subprocess
import psutil

# Funciones Auxiliares

# Funcion para convertir coordenas normalizadas a la resolucion de la imagen
def convertir_coordenada(coordenada_x , coordenada_y, width, height):
    x = int(coordenada_x * width)
    y = int(coordenada_y * height)
    return x, y

# Funcion para mapear las coordenadas a la pantalla
def mapeo_coordenadas(coordenada_x , coordenada_y, screen_width, screen_height):
    screen_x = int(coordenada_x * screen_width)
    screen_y = int(coordenada_y * screen_height)
    return screen_x, screen_y

# Funcion para calcular la distancia entre dos puntos
def distancia_entre_puntos(x1, y1, x2, y2):
    distancia = ((x2 - x1)**2 + (y2 - y1)**2)**0.5
    return distancia

# Funcion para verificar si un proceso esta activo. Debe recibir una string y devuelve un booleano.
def proceso_activo(proceso):
    #proceso.lower()
    for process in psutil.process_iter(['name']):
        try:
            if proceso in process.info['name']:
                print(True)
                return True
        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            pass
    print(False)
    return False

# Funcion para cerrar un proceso. Debe recibir una string con el nombre del proceso.
def cerrar_proceso(proceso):
    #proceso = proceso.lower()
    for process in psutil.process_iter(['name']):
        try:
            process_name = process.info['name']
            if proceso in process_name:
                process.terminate()
                process.wait()
                return True
        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            pass
    return False    
    

In [2]:
# Codigo para las ejecuciones de las macros. Se ha visto que se deben usar hilos de ejecución diferentes.
import pyautogui
import threading
import time
import pydirectinput

# Funciones para las macros
def macro_2_1():
    with pyautogui.hold('win'):
        pyautogui.press(['1'])
    print("macro21 exe")

def macro_2_2():
    with pyautogui.hold('win'):
        pyautogui.press('1')
    time.sleep(1)
    with pyautogui.hold('alt'):
        pyautogui.press('f4')
    print("macro22 exe")



# Funcion para ejecutar las macros
def ejecutar_macro(macro):
    match macro:
        case "macro_2_1":
            print("Ejecutando macro 2_1")
            hilo21 = threading.Thread(target=macro_2_1)
            hilo21.start()
            print("Windows + 1 abierto")
        case "macro_2_2":
            print("Ejecutando macro 2_2")
            hilo22 = threading.Thread(target=macro_2_2)
            hilo22.start()
            print("Windows + 1 cerrado")

In [ ]:
import cv2
import mediapipe as mp
import pyautogui
import subprocess
import psutil
import time
import pydirectinput

# Inicializamos pyautogui
pyautogui.FAILSAFE = False

# Inicializar MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

# Input Camera
vid = cv2.VideoCapture(0, cv2.CAP_DSHOW)

# Obtenemos la resolución de la pantalla
screen_width, screen_height = pyautogui.size()

# Constantes
width = 960
height = 720

# Variables. Contadores y Flags auxiliares.
# Flag para el cambio de funcionalidad de la mano izquierda
flag = True
# Contador de frames
frame_count = 0
# Contador procesos
proc_counter = 0
# Flag para tratamiento de procesos
proc_switch = True
# Contador para Macro 2
macro2_aux = 0


# Bucle de ejecución
while True:
    # Fotograma de la cámara
    ret, frame = vid.read()
    if not ret:
        break

    frame_count += 1

    # Saltar fotogramas para mejorar rendimiento (cada 2 fotogramas)
    if frame_count % 2 != 0:
        continue

    # Cambiar tamaño del fotograma
    frame = cv2.resize(frame, (width, height))
    # Voltear el fotograma
    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape  

    # Convertir el color BGR a RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Procesar el fotograma
    result = hands.process(rgb_frame)

    # Procesamiento
    if result.multi_hand_landmarks:
        for idx, hand_landmarks in enumerate(result.multi_hand_landmarks):
            # Dibujar los puntos clave de la mano en la imagen
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            # Obtener la información de la mano (derecha o izquierda)
            handedness = result.multi_handedness[idx].classification[0].label

            # Obtenemos los landmark 4(pulgar) , 8(indice), 12(corazón o medio), 16(anular), 20(menique) y 0 base de la palma
            dedo_pulgar = hand_landmarks.landmark[4] 
            dedo_indice = hand_landmarks.landmark[8]
            dedo_medio = hand_landmarks.landmark[12]
            dedo_anular = hand_landmarks.landmark[16]
            dedo_menique = hand_landmarks.landmark[20]
            base_palma = hand_landmarks.landmark[0]

            # Pasamos a coordenadas de la pantalla
            x_pulgar, y_pulgar = convertir_coordenada(dedo_pulgar.x, dedo_pulgar.y, w, h)
            x_indice, y_indice = convertir_coordenada(dedo_indice.x, dedo_indice.y, w, h)       
            x_medio, y_medio = convertir_coordenada(dedo_medio.x, dedo_medio.y, w, h)
            x_anular, y_anular = convertir_coordenada(dedo_anular.x, dedo_anular.y, w, h)
            x_menique, y_menique = convertir_coordenada(dedo_menique.x, dedo_menique.y, w, h)
            x_palma, y_palma = convertir_coordenada(base_palma.x, base_palma.y, w, h)

            # Calcular escala de la mano y el umbral para el clic
            escala_mano = distancia_entre_puntos(x_palma, y_palma, x_indice, y_indice)
            umbral_click = 0.2 * escala_mano # Ajustar el umbral

            # Si la mano es derecha:
            if handedness == 'Right':
                # Movemos el raton con el dedo índice
                # Mapeamos las coordenadas a la pantalla
                screen_x_indice, screen_y_indice = mapeo_coordenadas(dedo_indice.x, dedo_indice.y, screen_width, screen_height)
                # Marcamos y movemos el cursor a la posición del dedo índice
                cv2.circle(frame, (x_indice, y_indice), 10, (0, 255, 0), cv2.FILLED)
                # Movemos el cursor a la posición del dedo índice
                pyautogui.moveTo(screen_x_indice, screen_y_indice)

                # Comprobamos si los dedos están cerca para hacer clic
                # Calcular la distancia euclidiana entre el pulgar y el corazon
                distancia_pulgar_medio = distancia_entre_puntos(x_pulgar, y_pulgar, x_medio, y_medio)
                distancia_pulgar_anular = distancia_entre_puntos(x_pulgar, y_pulgar, x_anular, y_anular)
                distancia_pulgar_menique = distancia_entre_puntos(x_pulgar, y_pulgar, x_menique, y_menique)

                # Si la distancia es menor que un umbral, hacemos clic
                # Están los break para mejorar el rendimiento.
                
                if distancia_pulgar_menique < umbral_click:
                    if flag == True:
                        flag = False
                    else:
                        flag = True 
                    print(flag)  

                if flag == True:
                    if distancia_pulgar_medio < umbral_click:
                        # Macro 1: Abrir y cerrar teclado virtual
                        if proc_switch == True:
                            if proceso_activo("osk.exe"):            
                                cerrar_proceso("osk.exe")
                                print("Teclado virtual cerrado")
                                proc_switch = False
                                proc_counter = frame_count
                            else:
                                subprocess.Popen("osk.exe", shell=True)
                                print("Teclado virtual abierto")
                                proc_switch = False
                                proc_counter = frame_count
                        elif proc_counter + 15 < frame_count:
                            proc_switch = True
                    elif distancia_pulgar_anular < umbral_click:
                        # Macro 2 Abrir y cerrar "WINDOWS + 1"
                        if proc_switch == True:
                            if proceso_activo("CalculatorApp.exe"):            
                                cerrar_proceso("CalculatorApp.exe")
                                print("Calculadora cerrado")
                                proc_switch = False
                                proc_counter = frame_count
                            else:
                                subprocess.Popen("calc.exe", shell=True)
                                print("Calculadora abierto")
                                proc_switch = False
                                proc_counter = frame_count
                        elif proc_counter + 15 < frame_count:
                            proc_switch = True
                else:
                    if distancia_pulgar_medio < umbral_click:
                        # Macro 3: Abrir y cerrar calculadora
                        pass
                    elif distancia_pulgar_anular < umbral_click:
                        #macro 4
                        pass
                            
            # Si la mano es izquierda:
            elif handedness == 'Left':
                # Comprobamos si los dedos están cerca para hacer clic
                # Calcular la distancia euclidiana entre el pulgar y el corazon
                distancia_pulgar_indice = distancia_entre_puntos(x_pulgar, y_pulgar, x_indice, y_indice)
                distancia_pulgar_medio = distancia_entre_puntos(x_pulgar, y_pulgar, x_medio, y_medio)
                distancia_pulgar_anular = distancia_entre_puntos(x_pulgar, y_pulgar, x_anular, y_anular)
                distancia_pulgar_menique = distancia_entre_puntos(x_pulgar, y_pulgar, x_menique, y_menique)

                # Si la distancia es menor que un umbral, hacemos clic
                if distancia_pulgar_indice < umbral_click:
                    pyautogui.click()
                elif distancia_pulgar_medio < umbral_click:
                    pyautogui.click(button='right')

                # Comprobamos si el flag esta activado
                if flag == True:
                    if distancia_pulgar_anular < umbral_click:
                        # Macro 5
                        pass
                    elif distancia_pulgar_menique < umbral_click:
                        # Macro 6
                        pass
                else:
                    # Si la distancia es menor que un umbral, hacemos clic
                    if distancia_pulgar_anular < umbral_click:
                        # Macro 7
                        pass
                    elif distancia_pulgar_menique < umbral_click:
                        # Macro 8
                        pass

    # Muestra fotograma
    cv2.imshow('Vid', frame)

    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break
  
# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()